In [1]:
%pip install openai instructor

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.0 kB)
  Using cached charset_normalizer-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.1 MB/s eta 0:00:00
Using cached jinja2-3.1.5-py3-none-any.whl (134 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.1 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached certifi-2

In [1]:
import textwrap
import builtins

def wprint(*args, width=70, **kwargs):
    """
    Custom print function that wraps text to a specified width.

    Args:
    *args: Variable length argument list.
    width (int): The maximum width of wrapped lines.
    **kwargs: Arbitrary keyword arguments.
    """
    wrapper = textwrap.TextWrapper(width=width)

    # Process all arguments to make sure they are strings and wrap them
    wrapped_args = [wrapper.fill(str(arg)) for arg in args]

    # Call the built-in print function with the wrapped text
    builtins.print(*wrapped_args, **kwargs)

In [2]:
import openai
import instructor
from getpass import getpass
from openai import Client
client = Client(api_key = "XXXX")


In [3]:
import time

def get_completion(message, agent, funcs, thread):
    """
    Executes a thread based on a provided message and retrieves the completion result.

    This function submits a message to a specified thread, triggering the execution of an array of functions
    defined within a func parameter. Each function in the array must implement a `run()` method that returns the outputs.

    Parameters:
    - message (str): The input message to be processed.
    - agent (OpenAI Assistant): The agent instance that will process the message.
    - funcs (list): A list of function objects, defined with the instructor library.
    - thread (Thread): The OpenAI Assistants API thread responsible for managing the execution flow.

    Returns:
    - str: The completion output as a string, obtained from the agent following the execution of input message and functions.
    """

    # create new message in the thread
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=message
    )

    # run this thread
    run = client.beta.threads.runs.create(
      thread_id=thread.id,
      assistant_id=agent.id,
    )

    while True:
      # wait until run completes
      while run.status in ['queued', 'in_progress']:
        run = client.beta.threads.runs.retrieve(
          thread_id=thread.id,
          run_id=run.id
        )
        time.sleep(1)

      # function execution
      if run.status == "requires_action":
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_outputs = []
        for tool_call in tool_calls:
          wprint('\033[31m' + str(tool_call.function), '\033[0m')
          # find the tool to be executed
          func = next(iter([func for func in funcs if func.__name__ == tool_call.function.name]))

          try:
            # init tool
            func = func(**eval(tool_call.function.arguments))
            # get outputs from the tool
            output = func.run()
          except Exception as e:
            output = "Error: " + str(e)

          wprint(f"\033[33m{tool_call.function.name}: ", output, '\033[0m')
          tool_outputs.append({"tool_call_id": tool_call.id, "output": output})

        # submit tool outputs
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
      # error
      elif run.status == "failed":
        raise Exception("Run Failed. Error: ", run.last_error)
      # return assistant message
      else:
        messages = client.beta.threads.messages.list(
          thread_id=thread.id
        )
        message = messages.data[0].content[0].text.value
        return message

In [5]:
from openai import OpenAI


code_assistant_funcs = []

assistant = client.beta.assistants.create(
  name='Code Assistant Agent',
  instructions="As a top-tier web developement Cybersecurity AI, you are adept at analyzing potential vulnerabilities in API calls. You will analyze the Website Source Code file attached for exposure of sensitive data, security vulnerabilities, and compliance with best practices (e.g., OWASP, GDPR). Find all possible vulnerabilities and exploits while prioritize glaring issues. ",
  model="gpt-4o-mini",
  tools=[{"type": "file_search"}]
)

In [4]:
vector_store = client.beta.vector_stores.create(name="Website Source File")

# Ready the files for upload to OpenAI
file_paths = ["/home/albert818/Documents/hackathon_deep/hack2lytics/AdminPage.ts"]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [ ]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("/home/albert818/Documents/hackathon_deep/hack2lytics/AdminPage.ts", "rb"), purpose="assistants"
)

# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Are there any cybersecurity issues with this typescript source code for my",
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)

# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_67ba13bfeda08191a45fd7e6f8d00108'])


In [28]:
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI


class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))

# Then, we use the stream SDK helper
# with the EventHandler class to create the Run
# and stream the response.

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as PI Man. The user has a premium account.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()


assistant > Please upload the TypeScript source code file so I can help you analyze it for potential cybersecurity issues.



In [ ]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [ ]:
from openai import OpenAI
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a cybersecurity expert specializing in web development and security. Analyze the following request for exposure of sensitive data, security vulnerabilities, and compliance with best practices (e.g., OWASP, GDPR). Provide recommendations for improving security."},
        {"role": "user", "content": """
         REQUEST
{"query":"query ListEvents($filter: ModelEventFilterInput, $limit: Int, $nextToken: String) {\n  listEvents(filter: $filter, limit: $limit, nextToken: $nextToken) {\n    items {\n      id\n      name\n      description\n      status\n      requireRSVP\n      canRSVP\n      start\n      end\n      location\n      points\n      checkInCode\n      createdAt\n      updatedAt\n      _version\n      _deleted\n      _lastChangedAt\n      __typename\n    }\n    nextToken\n    startedAt\n    __typename\n  }\n}\n","variables":{"filter":{"_deleted":{"ne":true}},"limit":1000}}

RESPONSE
{"data":{"listEvents":{"items":[{"id":"09d73bcd-6ef0-4a17-ad86-c4f100384ae2","name":"Sunday Breakfast","description":"Blue Donkey Coffee and Bagels","status":false,"requireRSVP":false,"canRSVP":false,"start":"2025-02-23T14:30:00.000Z","end":"2025-02-23T15:30:00.000Z","location":"Klaus 1116W","points":0,"checkInCode":"Bagul","createdAt":"2025-02-20T20:36:35.504Z","updatedAt":"2025-02-21T04:06:32.338Z","_version":2,"_deleted":null,"_lastChangedAt":1740110792378,"__typename":"Event"},{"id":"a583ccdb-16d6-47dc-b4dd-51c8f12dd3ae","name":"Saturday Lunch","description":"Moes Burritos\n** ALL ITEMS CONTAIN GLUTEN**","status":false,"requireRSVP":false,"canRSVP":false,"start":"2025-02-22T18:00:00.000Z","end":"2025-02-22T19:00:00.000Z","location":"Klaus 1116W","points":0,"checkInCode":"lonch","createdAt":"2025-02-20T20:19:47.857Z","updatedAt":"2025-02-21T04:04:28.124Z","_version":4,"_deleted":null,"_lastChangedAt":1740110668170,"__typename":"Event"},{"id":"f6f5d6bb-9c35-4f58-9e51-67c3033b2ae2","name":"Sponsor Fair","description":"ChitChat!","status":false,"requireRSVP":false,"canRSVP":false,"start":"2025-02-22T00:15:00.000Z","end":"2025-02-22T01:15:00.000Z","location":"Klaus Atrium","points":0,"checkInCode":"vibes","createdAt":"2025-02-20T02:25:49.629Z","updatedAt":"2025-02-20T02:29:20.894Z","_version":3,"_deleted":null,"_lastChangedAt":1740018560942,"__typename":"Event"}
    """}
    ]
)

print(response.choices[0].message.content)
